In [1]:
import scipy.integrate as spi
import numpy as np
import pylab as pl
import numpy as np
#import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy import integrate
from pylab import *
from scipy.optimize import curve_fit
import numpy as np
from scipy import optimize
import pandas as pd

In [2]:
df = pd.read_csv('../data/estados.csv')

In [3]:
df.state.unique()

array(['TOTAL', 'SP', 'RJ', 'ES', 'BA', 'DF', 'AL', 'MG', 'RS', 'PE',
       'PR', 'SC', 'GO', 'RN', 'AM', 'MS', 'SE', 'CE', 'AC', 'TO', 'PA',
       'PB', 'PI', 'RO', 'MA', 'MT'], dtype=object)

In [ ]:
dfUF = []
for value in df.state.unique():
    data =df[df['state']==value]['totalCases']
    data = np.array(data)
    dfUF.append(data)

In [ ]:
popUF = [45919049, 210147125,17264943,4018650, 14873064,3015268,3337357,21168791,11377239,9557071,11433957,7164788,7018354,
        3506853,4144597,2778986,2298696,9132078]

In [ ]:
dfUF_norm = []
for i in range(0,len(dfUF)):
    data = dfUF[0]/popUF[0]
    dfUF_norm.append(data)

# SIR

In [ ]:
# Initial conditions
I0 = 1/popUF[0]
S0= 1 -I0

UF = df.state.unique()[0]

print('condições iniciais para ' + df.state.unique()[0])

Time = range(0,len(dfUF_norm[0]))
DayFinal = len(dfUF_norm[0])

data = dfUF_norm[0]

In [ ]:
gamma = 1/14

In [ ]:
#for n in Time:
#    print(n)
data

In [ ]:
#=EXAMPLE+DEFINING THE Solution of DF-EQ, TRIAL run NEEDS BETA and so on======================================================
def eq(par,initial_cond,start_t,end_t,incr):
    #-time-grid-----------------------------------
    t  = np.linspace(start_t, end_t, num=incr, endpoint=True)
    def funct(y,t):
        S=y[0]
        I=y[1]
        R=y[2]
        beta, gamma = par
        f0 = - beta * S * I        
        f1 = beta * S * I - gamma * I
        f2 = gamma * I
        return [f0, f1, f2]
     #integrate------------------------------------
    ds = integrate.odeint(funct,initial_cond,t)
    return (ds[:,0],ds[:,1],ds[:,2],t)
#=======================================================

In [ ]:
ND=100.0
beta = 0.30
Param=(beta,gamma)
#S0=1-1e-6          # Initial condition need to be defined 
#I0=1e-6
INPUT = [S0, I0, 0.0] # R(0) = 0
         # initial condition vector

Sf, If, Rf, Days = eq(Param,INPUT,1,ND,800)  #1time, steps

pl.plot(Days,If)
pl.plot(Days,Sf)
pl.plot(Days,Rf)
pl.plot(data, 'ob')
pl.show()

In [ ]:
#Fitting of FULL MODEL
def fitFunc(x,beta):
    Param = (beta, gamma)
    INPUT = [S0, I0, 0.0]
    return eq(Param,INPUT,1,DayFinal,DayFinal)[1][x]

p0 = (beta)
popt, pcov = curve_fit(fitFunc, Time, data, p0)
popt, pcov

In [ ]:
#BOOTSTRAP FITTING!
def fit_bootstrap(p0, datax, datay, function, yerr_systematic=0.0):

    errfunc = lambda p, x, y: function(x,p) - y

    # Fit first time
    pfit, perr = optimize.leastsq(errfunc, p0, args=(datax, datay), full_output=0)


    # Get the stdev of the residuals
    residuals = errfunc(pfit, datax, datay)
    sigma_res = np.std(residuals)

    sigma_err_total = np.sqrt(sigma_res**2 + yerr_systematic**2)

    # 100 random data sets are generated and fitted
    ps = []
    for i in range(2000):

        randomDelta = np.random.normal(0., sigma_err_total, len(datay))
        randomdataY = datay + randomDelta

        randomfit, randomcov = \
            optimize.leastsq(errfunc, p0, args=(datax, randomdataY),\
                             full_output=0)

        ps.append(randomfit) 

    ps = np.array(ps)
    mean_pfit = np.mean(ps,0)

    # You can choose the confidence interval that you want for your
    # parameter estimates: 
    Nsigma = 2. # 1sigma gets approximately the same as methods above
                # 1sigma corresponds to 68.3% confidence interval
                # 2sigma corresponds to 95.44% confidence interval
    err_pfit = Nsigma * np.std(ps,0) 

    pfit_bootstrap = mean_pfit
    perr_bootstrap = err_pfit
    return pfit_bootstrap, perr_bootstrap 

In [ ]:
#COMPARE BOOTSTRAP AND CURVE FIT!
BFIT, BError = fit_bootstrap(p0, Time, data, fitFunc, yerr_systematic=0.0000000)
BFIT, BError

In [ ]:
#====PLOTS===================================================
def eq(par,initial_cond,start_t,end_t,incr):
    #-time-grid-----------------------------------
    t  = np.linspace(start_t, end_t, incr)
    def funct(y,t):
        S=y[0]
        I=y[1]
        R=y[2]
        beta, gamma = par
        f0 = - beta * S * I        
        f1 = beta * S * I - gamma * I
        f2 = gamma * I
        return [f0, f1, f2]
     #integrate------------------------------------
    ds = integrate.odeint(funct,initial_cond,t)
    return (ds[:,0],ds[:,1],ds[:,2],t)
#=======================================================
     
#parameters  
#beta= bUp      
gamma=1.0/14.0
#TS=0.1
ND=100.0
Param=(popt[0],gamma)
ParamErr1=(BFIT+BError,gamma)
ParamErr2=(BFIT-BError,gamma)
#S0=1-1e-6          # Initial condition need to be defined 
#I0=1e-6
INPUT = [S0,I0, 0.0] # R(0) = 0
         # initial condition vector

Sf, If, Rf, Days = eq(Param,INPUT,0,ND,101)  #1time, steps
Sf1, If1, Rf1, Days1 = eq(ParamErr1,INPUT,0,ND,101) 
Sf2, If2, Rf2, Days2 = eq(ParamErr2,INPUT,0,ND,101) 

figure(figsize=(8, 6), dpi=100, facecolor='w', edgecolor='k')
#fig, ax = pl.subplots()
pl.plot(Days,If, '-')
pl.plot(data, 'or')
pl.fill_between(Days, If1, If2, alpha=0.2)
#pl.errorbar(Time, df_BA, xerr=0.0, yerr=0.0000000, color='r', fmt='o',  ecolor='g',capsize=2,barsabove=True,elinewidth=0.8,)
pl.xlim(0.0,25)
pl.ylim(0,0.00002)
pl.show()

In [ ]:
data*popUF[0]

In [ ]:
#df[df['state'] == 'SP']

In [ ]:
cases = pd.Series(data*popUF[0],
                     index = pd.date_range('02/25/2020' ,
                                           periods = 23,
                                           freq = 'D'))

pred = pd.Series(If*popUF[0],
                index = pd.date_range('02/25/2020' ,
                                           periods = 101,
                                           freq = 'D'))

In [ ]:
cases = cases.to_frame()
pred = pred.to_frame()

In [ ]:
pred = pred.rename(columns={0: 'total_predito'})
cases = cases.rename(columns={0: 'total_cases'})

In [ ]:
pred['total_predito'] = pred['total_predito'].astype(int)

In [ ]:
pred.join(cases)

In [ ]:
# Zoom to see the data
pl.semilogy(Days,If, '-k', label= ' SIR: $\\beta =$' + str(round(BFIT[0],3))) # monolog
pl.semilogy(data, 'ob')
pl.legend(loc=0)
pl.xlim(1,30)
pl.xlabel('Tempo (dias)',fontsize = 14)
pl.ylabel('Infectados',fontsize = 14)
pl.text(2.3, 0.001, '$R_0$ = ' + str(round(BFIT[0]/gamma,2)) + ' (número basico de reprodução) em ' + UF +
         ', \n $\gamma = $' + str(round(gamma,3)) + \
         ' (' + str(round(1/gamma)) + ' dias de recuperação)', style='italic',
    fontsize=12, fontname='Ubuntu',
    bbox={'facecolor':'grey', 'alpha':0.1, 'pad':10})
#pl.savefig('SIR_BA_analise_fit.pdf')
pl.show()

In [ ]:
#fig, ax = pl.subplots()
figure(figsize=(8, 6), dpi=100, facecolor='w', edgecolor='k')
pl.plot(Days,If, '-b')
pl.plot(Days,Sf, '-r')
pl.plot(Days,Rf, '-g')
pl.plot(Time, data, 'ob')
pl.fill_between(Days, If1, If2, alpha=0.2,color='b')
pl.fill_between(Days, Sf1, Sf2, alpha=0.2,color='r')
pl.fill_between(Days, Rf1, Rf2, alpha=0.2,color='g')
#pl.errorbar(Time, data, xerr=0.0,yerr=0.0000001,  color='r', fmt='o',  ecolor='g',capsize=2,barsabove=True,elinewidth=0.8,)
#pl.errorbar(Time, data, xerr=0.0,yerr=0.0000001,  color='r', fmt='o',  ecolor='g',capsize=2,barsabove=True,elinewidth=0.8,)
pl.show()


In [ ]:
figure(figsize=(10, 6), dpi=100, facecolor='w', edgecolor='k')
#pl.loglog(t_range,RESLow[:,1], '--b', label='Infectious (Low bound)')
pl.plot(pred[0:33], 'or', label='Predito')
#pl.loglog(t_range,RESUp[:,1], '--b', label='Infectious (Upper bound)')
pl.plot(cases, 'ob', label='Dados reais')
pl.legend(loc=0)
#pl.loglog(time2,func(time2)/14873064, '-g')
#pl.xlim(0,28)
pl.xlabel('Tempo (dias)',fontsize = 14)
pl.xticks(rotation=45)
pl.ylabel('Número de Infectados',fontsize = 14)
pl.title('Predição de casos para '+ UF  + ': $R_0$ = ' + str(round(BFIT[0]/gamma,2)) + ' (número basico de reprodução)' +
         ', \n $\\beta$ = ' + str(round(BFIT[0],3)) +  ' (Taxa de transmissão per capita)' \
         ', \n $\gamma = $' + str(round(gamma,3)) + \
         ' (' + str(round(1/gamma)) + ' dias de infecciosidade)', fontsize = 15)
#pl.savefig('SIR_BA_predito_vs_data.pdf',dpi=1000)
pl.show()

In [ ]:
79 +45